In [1]:
import pandas as pd
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC
from sklearn.model_selection import KFold

In [28]:
###Laod data
df = pd.read_csv('Finalplfam_id_Multilabel_Salmonella_data.csv', dtype=str, index_col=0)

In [29]:
Labeldf = pd.read_csv('AMR_LAbel_Salmonella.csv',index_col=0) 

In [30]:
Ycolumns=Labeldf.columns.values
Ycolumns=Ycolumns.tolist()

In [31]:
Ycolumns

['genome_id',
 'genome_name',
 'taxon_id',
 'amoxicillin_clavulanic_acid',
 'ampicillin',
 'azithromycin',
 'cefoxitin',
 'ceftiofur',
 'chloramphenicol',
 'ciprofloxacin',
 'gentamicin',
 'kanamycin',
 'nalidixic_acid',
 'streptomycin',
 'sulfonamides',
 'tetracycline',
 'trimethoprim_sulphamethoxazole',
 'spectinomycin',
 'trimethoprim',
 'ceftazidime',
 'ceftriaxone',
 'sulfisoxazole',
 'netilmicin',
 'cephalosporins',
 'furazolidone']

In [32]:
Y=df[Ycolumns]
Y=Y.drop(columns=['genome_id', 'genome_name','taxon_id'])

X=df.drop(columns=Ycolumns)

In [33]:
Y=Y.fillna(-1)

In [34]:
Ycolumns=Y.columns.values
Ycolumns=Ycolumns.tolist()

In [35]:
###Data distribution check
for index, label in enumerate(Ycolumns):
    V= Y[label].value_counts()
    if(index==0):
        df_val_counts = pd.DataFrame(V)
    else:
        df_val_counts_t = pd.DataFrame(V)
        df_val_counts = df_val_counts.join(df_val_counts_t)
        
print (df_val_counts)
df_val_counts.to_csv('Input_Label_Supplementry_Sal.csv')

     amoxicillin_clavulanic_acid  ampicillin  azithromycin  cefoxitin  \
0                           1883      2247.0        2289.0       2070   
-1                          1589       167.0        1599.0       1498   
1                            401      1516.0          49.0        336   
0.5                           64         NaN           NaN         33   

     ceftiofur  chloramphenicol  ciprofloxacin  gentamicin  kanamycin  \
0         1863             2963           2733        2358     1027.0   
-1        1673              190            210        1251     2830.0   
1          400              770            627         314       80.0   
0.5          1                8            131          14        NaN   

     nalidixic_acid  ...  tetracycline  trimethoprim_sulphamethoxazole  \
0              2650  ...          1351                          2698.0   
-1              797  ...           982                           858.0   
1               260  ...          1601        

In [10]:
#X=X.drop(columns=['amoxicillin/clavulanic acid', 'aztreonam', 'cefepime', 'cefotaxime', 'piperacillin/tazobactam', 'sulfamethoxazole/trimethoprim', 'tobramycin'])

In [11]:
Ycolumns=Y.columns

In [12]:
Labellist=Ycolumns.tolist()

In [13]:
kfold = KFold(n_splits=5)
scorelist=[]

In [14]:
X=X.fillna(0)
X = X.apply(pd.to_numeric)
Y = Y.replace(to_replace=[0.5], value=[1])
Y = Y.replace(to_replace=['0.5'], value=[1])

In [15]:
def hamming_score(y_pred,y_true):
    i=0
    scorelist=[]
    for  amrtrue in y_true:
        match=0.0
        total=0.0
        #print(amrtrue, y_pred[i])
        j=0
        for trueentry in amrtrue:
            #print(trueentry)
            if (trueentry == y_pred[i][j]):
                #print ('match found')
                match=match+1
                total=total+1
            elif (not (np.isnan(trueentry))):
                total = total + 1
            j=j+1
        i=i+1
        #print (match, total, (match/total))
        scorelist.append(match/total)
    return np.mean(scorelist)

In [16]:
import numpy as np
def Modified_F1_Score(y_pred,y_true):
    #i=0
    scorelist=[]
    for  i,amrtrue in enumerate(y_true):
        TP,FP,TN,FN=0.00000001,0.00000001,0.00000001,0.00000001
        #j=0
        for j,trueentry in enumerate(amrtrue):
            if ((np.isnan(trueentry))):
                continue
            elif ((trueentry == 1) and (y_pred[i][j]==1)):
                TP=TP+1
            elif ((trueentry == 1) and (y_pred[i][j]==0)):
                FN=FN+1
            elif ((trueentry == 0) and (y_pred[i][j]==0)):
                TN=TN+1
            elif ((trueentry == 0) and (y_pred[i][j]==1)):
                FP=FP+1
            #j=j+1
        #i=i+1
        precision=TP/(TP+FP)
        recall=TP/(TP+FN)
        fscore=(2*precision*recall)/(precision+recall)
        scorelist.append(fscore)
    return np.mean(scorelist)

In [17]:
import matplotlib.pyplot as plt
import seaborn as sb

sb.set_context("talk")
def plot_coefficients(feature_names, coef, name, top_features=20):
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    #top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients =top_positive_coefficients# np.hstack([top_positive_coefficients])
    # create plot
    plt.figure(figsize=(10, 5))
    plt.title("Feature Importances Multi AMR for "+str(name), y=1.08)
    colors = ['crimson' if c < 0 else 'cornflowerblue' for c in coef[top_coefficients]]
    plt.bar(np.arange(1 * top_features), coef[top_coefficients], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(0, 1 * top_features), feature_names[top_coefficients], rotation=60, ha='right')
    plt.show()
    np.asarray(feature_names)[top_positive_coefficients]

In [18]:
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

In [19]:
####Base models
xgbmodel = XGBClassifier(eta=0.0625, max_depth=16, eval_metric='mlogloss', seed=20211028)
adaboostmodel = AdaBoostClassifier(n_estimators=100, random_state=0)
modellist=[xgbmodel,adaboostmodel]
namelist=['xgb','AdaBoost']

In [20]:
import time
times=[]
methods=[]

In [21]:
Y=Y.replace(to_replace=['IS','Not defined'], value=[np.nan,np.nan])

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance

In [ ]:
def BinaryRelevanceModel(X,Y,kfold, basemodel, name, times, methods):
    classifier = BinaryRelevance(basemodel)
    hammingscorelist=[]
    fscorelist=[]
    for train_index, test_index in kfold.split(X, Y):
        start = time.process_time()
        # split data into train/test sets
        x_train_tfidf = X.iloc[train_index]
        y_train_tfidf = Y.iloc[train_index]
        x_test_tfidf = X.iloc[test_index]
        y_test_tfidf = Y.iloc[test_index]
        y_train_tfidf=y_train_tfidf.fillna(0)
        y_train_tfidf = y_train_tfidf.apply(pd.to_numeric)
        y_train_tfidf=y_train_tfidf.astype(int)
        y_test_tfidf = y_test_tfidf.apply(pd.to_numeric)
        #y_test_tfidf=y_test_tfidf.astype(int)
        classifier.fit(x_train_tfidf.values, y_train_tfidf.values)
        # calculating test accuracy
        prediction = classifier.predict(x_test_tfidf.values)
        hammingscore=hamming_score(prediction.toarray(),y_test_tfidf.values)
        fscore=Modified_F1_Score(prediction.toarray(),y_test_tfidf.values)
        print(fscore, hammingscore)
        hammingscorelist.append(hammingscore)
        fscorelist.append(fscore)
        times = np.append(times, (time.process_time() - start))
        methods = np.append(methods, 'BR-'+name)
    print ('Base model for BR is {}'.format(name))
    print('Test Hamming accuracy for Binary relvance is {}'.format(np.mean(hammingscorelist)))
    print('Test F-score accuracy for Binary relvance is {}'.format(np.mean(fscorelist)))
    print('Hamming std'+str(np.std(hammingscorelist)))
    print('F-Score std'+str(np.std(fscorelist)))
    return np.mean(hammingscorelist),np.mean(fscorelist),np.std(hammingscorelist),np.std(fscorelist), times, methods


In [ ]:
i=0
for basemodel in modellist:
    hamming, fscore, hmstd, fstd, times, methods=BinaryRelevanceModel(X,Y,kfold,basemodel,namelist[i], times, methods)
    i=i+1

In [ ]:
#!pip install shap

In [22]:
print("Before removing low variance: ", X.shape)
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.01)
selector.fit_transform(X)
X = X[X.columns[selector.get_support()]].copy()
print("After removing low variance: ", X.shape)

Before removing low variance:  (3937, 16358)
After removing low variance:  (3937, 7063)


In [ ]:
!pip install imblearn

In [23]:
from RectifiedClassifierChain import RectifiedClassiferChain

In [24]:
Y

,amoxicillin_clavulanic_acid,ampicillin,cefoxitin,ceftiofur,chloramphenicol,ciprofloxacin,gentamicin,kanamycin,nalidixic_acid,streptomycin,sulfonamides,tetracycline,trimethoprim_sulphamethoxazole,spectinomycin,trimethoprim,ceftriaxone,sulfisoxazole,cephalosporins
0,1,1,1,1,0,0,0,0,0,0,NaN,0,0,NaN,NaN,1,0,NaN
1,NaN,1,NaN,NaN,1,0,0,0,0,1,1,1,NaN,1,0,NaN,NaN,NaN
2,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
3,1,1,0,1,0,0,1,NaN,0,1,NaN,1,0,NaN,NaN,1,1,NaN
4,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3932,1,1,1,1,0,0,0,1,0,0,NaN,1,0,NaN,NaN,1,1,NaN
3933,0,0,0,0,0,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,NaN
3934,0,0,NaN,NaN,0,0,0,0,1,0,NaN,1,NaN,NaN,0,NaN,NaN,NaN
3935,NaN,0,NaN,NaN,1,0,0,0,0,1,1,1,NaN,1,0,NaN,NaN,NaN


In [25]:
def RCCModelwithMR(X,Y,kfold, basemodel, name, times, methods,shapTotal, type=0):
    sim_all_df = pd.DataFrame()
    hammingscorelist=[]
    fscorelist=[]
    for train_index, test_index in kfold.split(X, Y):
        # split data into train/test sets
        start = time.process_time()
        x_train_tfidf = X.iloc[train_index]
        y_train_tfidf = Y.iloc[train_index]
        x_test_tfidf = X.iloc[test_index]
        y_test_tfidf = Y.iloc[test_index]
        x_train_tfidf=x_train_tfidf.reset_index(drop=True)
        y_train_tfidf=y_train_tfidf.reset_index(drop=True)
        y_train_tfidf = y_train_tfidf.apply(pd.to_numeric)
        #y_train_tfidf=y_train_tfidf.astype(int)
        y_test_tfidf = y_test_tfidf.apply(pd.to_numeric)
        classifier=RectifiedClassiferChain(basemodel,optimized=True,optimizedmethod='MissingRatio')
        classifier.trainRCC(x_train_tfidf, y_train_tfidf)
        # calculating test accuracy
        x_test_tfidf=x_test_tfidf.reset_index(drop=True)
        y_test_tfidf=y_test_tfidf.reset_index(drop=True)
        prediction = classifier.predictRCC(x_test_tfidf)
        hammingscore, fscore=classifier.Evaluate(y_test_tfidf,prediction)
        
        #hammingscore1=hamming_score(np.array(prediction),y_test_tfidf.values)
        #fscore1=Modified_F1_Score(np.array(prediction),y_test_tfidf.values)
        
        print(hammingscore,fscore)
        #print(hammingscore1,fscore1)
        hammingscorelist.append(hammingscore)
        fscorelist.append(fscore)
        times = np.append(times, (time.process_time() - start))
        methods = np.append(methods, 'RCC_MR-'+name)
        label_order=classifier.getOptimizedLabelOrder()
        print (label_order)
        if(type !=2):
            featuredf=classifier.getFeature(NoOfFeature=100,type=type,full=True)
            #featuredf.to_csv('Test.csv')
            sim_all_df = pd.concat([sim_all_df, featuredf], ignore_index=True)
        if(type==3):
            shapValue=classifier.getShapFeatures()
            shapTotal.append(shapValue)
    print ('Base model for RCC is {}'.format(name))
    print('Test Hamming accuracy for RCC is {}'.format(np.mean(hammingscorelist)))
    print('Test F-score accuracy for RCC is {}'.format(np.mean(fscorelist)))
    print('Hamming std'+str(np.std(hammingscorelist)))
    print('F-Score std'+str(np.std(fscorelist)))
    if(type !=2):
        sim_all_df_T = sim_all_df.transpose().copy()
        sim_all_df_T["feature_weight_sum"] = sim_all_df_T.apply(lambda x: abs(x).sum(), axis=1)
        sim_all_df_T_top = sim_all_df_T.sort_values("feature_weight_sum", ascending=False)[:30]
        #sim_all_df_T_top.to_csv('Test.csv')
    else:
        sim_all_df_T_top=sim_all_df.copy()
    return np.mean(hammingscorelist),np.mean(fscorelist),sim_all_df_T_top, times, methods, shapTotal

In [26]:
modellist=[xgbmodel]
namelist=['xgb']

In [27]:
i=0
TotShape=[]
for basemodel in modellist:
    if(namelist[i] =='AdaBoost'):
        type=1
    elif(namelist[i] =='xgb'):
        type=3
    elif(namelist[i] =='Gaussian'):
        type=2
    else:
        type=0
    hamming, fscore, featuredf, times, methods, TotShape=RCCModelwithMR(X,Y,kfold,basemodel,namelist[i], times, methods, TotShape, type)
    if(type!=2):
        featuredf.to_csv('sim_all_df_T_RCC_MR_Sal'+str(namelist[i])+'.csv')
        #print(featuredf)
        plot_coefficients(list(featuredf.index),featuredf["feature_weight_sum"],namelist[i],30)
    i=i+1

/users/mt01034/Documents/MTM/NewData/ML4AMR/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/users/mt01034/Documents/MTM/NewData/ML4AMR/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/users/mt01034/Documents/MTM/NewDa

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [ ]:
def RCCModelwithCP(X,Y,kfold, basemodel, name, times, methods,shapTotal, type=0):
    sim_all_df = pd.DataFrame()
    hammingscorelist=[]
    fscorelist=[]
    for train_index, test_index in kfold.split(X, Y):
        # split data into train/test sets
        start = time.process_time()
        x_train_tfidf = X.iloc[train_index]
        y_train_tfidf = Y.iloc[train_index]
        x_test_tfidf = X.iloc[test_index]
        y_test_tfidf = Y.iloc[test_index]
        x_train_tfidf=x_train_tfidf.reset_index(drop=True)
        y_train_tfidf=y_train_tfidf.reset_index(drop=True)
        y_train_tfidf = y_train_tfidf.apply(pd.to_numeric)
        #y_train_tfidf=y_train_tfidf.astype(int)
        y_test_tfidf = y_test_tfidf.apply(pd.to_numeric)
        classifier=RectifiedClassiferChain(basemodel,optimized=True,optimizedmethod='ConditionalProb')
        classifier.trainRCC(x_train_tfidf, y_train_tfidf)
        # calculating test accuracy
        x_test_tfidf=x_test_tfidf.reset_index(drop=True)
        y_test_tfidf=y_test_tfidf.reset_index(drop=True)
        prediction = classifier.predictRCC(x_test_tfidf)
        hammingscore, fscore=classifier.Evaluate(y_test_tfidf,prediction)
        
        #hammingscore1=hamming_score(np.array(prediction),y_test_tfidf.values)
        #fscore1=Modified_F1_Score(np.array(prediction),y_test_tfidf.values)
        
        print(hammingscore,fscore)
        #print(hammingscore1,fscore1)
        hammingscorelist.append(hammingscore)
        fscorelist.append(fscore)
        times = np.append(times, (time.process_time() - start))
        methods = np.append(methods, 'RCC_CE-'+name)
        label_order=classifier.getOptimizedLabelOrder()
        print (label_order)
        if(type !=2):
            featuredf=classifier.getFeature(NoOfFeature=100,type=type,full=True)
            #featuredf.to_csv('Test.csv')
            sim_all_df = pd.concat([sim_all_df, featuredf], ignore_index=True)
        if(type==3):
            shapValue=classifier.getShapFeatures()
            shapTotal.append(shapValue)
    print ('Base model for RCC is {}'.format(name))
    print('Test Hamming accuracy for RCC is {}'.format(np.mean(hammingscorelist)))
    print('Test F-score accuracy for RCC is {}'.format(np.mean(fscorelist)))
    print('Hamming std'+str(np.std(hammingscorelist)))
    print('F-Score std'+str(np.std(fscorelist)))
    if(type !=2):
        sim_all_df_T = sim_all_df.transpose().copy()
        sim_all_df_T["feature_weight_sum"] = sim_all_df_T.apply(lambda x: abs(x).sum(), axis=1)
        sim_all_df_T_top = sim_all_df_T.sort_values("feature_weight_sum", ascending=False)[:30]
        #sim_all_df_T_top.to_csv('Test.csv')
    else:
        sim_all_df_T_top=sim_all_df.copy()
    return np.mean(hammingscorelist),np.mean(fscorelist),sim_all_df_T_top, times, methods, shapTotal

In [ ]:
i=0
TotShape=[]
for basemodel in modellist:
    if(namelist[i] =='AdaBoost'):
        type=1
    elif(namelist[i] =='xgb'):
        type=3
    elif(namelist[i] =='Gaussian'):
        type=2
    else:
        type=0
    hamming, fscore, featuredf, times, methods, TotShape=RCCModelwithCP(X,Y,kfold,basemodel,namelist[i], times, methods, TotShape, type)
    if(type!=2):
        featuredf.to_csv('sim_all_df_T_RCC_CP_Sal'+str(namelist[i])+'.csv')
        #print(featuredf)
        plot_coefficients(list(featuredf.index),featuredf["feature_weight_sum"],namelist[i],30)
    i=i+1

In [ ]:
from StackedClassifierChain import StackedClassifierChain 

In [ ]:
def SCCModelwithSSC(X,Y,kfold, basemodel, name, times, methods, type=0):
    sim_all_df = pd.DataFrame()
    hammingscorelist=[]
    fscorelist=[]
    for train_index, test_index in kfold.split(X, Y):
        # split data into train/test sets
        start = time.process_time()
        x_train_tfidf = X.iloc[train_index]
        y_train_tfidf = Y.iloc[train_index]
        x_test_tfidf = X.iloc[test_index]
        y_test_tfidf = Y.iloc[test_index]
        x_train_tfidf=x_train_tfidf.reset_index(drop=True)
        y_train_tfidf=y_train_tfidf.reset_index(drop=True)
        y_train_tfidf=y_train_tfidf.fillna(0)
        y_train_tfidf = y_train_tfidf.apply(pd.to_numeric)
        y_train_tfidf=y_train_tfidf.astype(int)
        y_test_tfidf = y_test_tfidf.apply(pd.to_numeric)
        classifier=StackedClassifierChain(basemodel)
        classifier.trainSCC(x_train_tfidf, y_train_tfidf)
        # calculating test accuracy
        x_test_tfidf=x_test_tfidf.reset_index(drop=True)
        y_test_tfidf=y_test_tfidf.reset_index(drop=True)
        prediction = classifier.predictSCC(x_test_tfidf)
        hammingscore, fscore=classifier.Evaluate(y_test_tfidf,prediction)
        
        #hammingscore1=hamming_score(np.array(prediction),y_test_tfidf.values)
        #fscore1=Modified_F1_Score(np.array(prediction),y_test_tfidf.values)
        
        print(hammingscore,fscore)
        #print(hammingscore1,fscore1)
        hammingscorelist.append(hammingscore)
        fscorelist.append(fscore)
        times = np.append(times, (time.process_time() - start))
        methods = np.append(methods, 'SCC-'+name)
        if(type !=2):
            featuredf=classifier.getFeature(NoOfFeature=100,type=type,full=True)
            #featuredf.to_csv('Test.csv')
            sim_all_df = pd.concat([sim_all_df, featuredf], ignore_index=True)
    print ('Base model for SCC is {}'.format(name))
    print('Test Hamming accuracy for SCC is {}'.format(np.mean(hammingscorelist)))
    print('Test F-score accuracy for SCC is {}'.format(np.mean(fscorelist)))
    print('Hamming std'+str(np.std(hammingscorelist)))
    print('F-Score std'+str(np.std(fscorelist)))
    if(type !=2):
        sim_all_df_T = sim_all_df.transpose().copy()
        sim_all_df_T["feature_weight_sum"] = sim_all_df_T.apply(lambda x: abs(x).sum(), axis=1)
        sim_all_df_T_top = sim_all_df_T.sort_values("feature_weight_sum", ascending=False)[:30]
        sim_all_df_T_top.to_csv('SCCTest.csv')
    else:
        sim_all_df_T_top=sim_all_df.copy()
    return np.mean(hammingscorelist),np.mean(fscorelist),sim_all_df_T_top, times, methods

In [ ]:
i=0
for basemodel in modellist:
    if(namelist[i] =='randomforest' or namelist[i] =='xgb' or namelist[i]=='AdaBoost'):
        type=1
    elif(namelist[i] =='Gaussian'):
        type=2
    else:
        type=0
    hamming, fscore, featuredf, times, methods=SCCModelwithSSC(X,Y,kfold,basemodel,namelist[i], times, methods,type)
    if(type!=2):
        featuredf.to_csv('sim_all_df_T_SCC_SSC_Sal'+str(namelist[i])+'.csv')
        #print(featuredf)
        plot_coefficients(list(featuredf.index),featuredf["feature_weight_sum"],namelist[i],30)
    i=i+1

In [ ]:
'''
XGB -A tree depth of 16 , 10-mers were used to build models from assembled genomes to ensure that all models would fit in memory
 learning rate was set to 0.0625
    column and row subsampling was set to 1.0 
     number of rounds of boosting was limited to 1000
'''

In [ ]:
'''
AdaBoost
K=31
'''

In [ ]:
'''
##https://journals.asm.org/doi/full/10.1128/JCM.01260-18
https://github.com/PATRIC3/mic_prediction
    
###https://github.com/TahaAslani/AAk-mer
https://www.mdpi.com/2079-7737/9/11/365/htm

##https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008319
'''

In [ ]:
import shap
#explainer=shap.TreeExplainer(rf_classifier)
rf_shap_values = shap.KernelExplainer(rf_classifier.predict,X_test)

In [ ]:
shap.initjs()
prediction=rf_classifier.predict(X_test)

In [ ]:
shap.force_plot(rf_shap_values.expected_value,shap_values,X_test)

In [ ]:
KErnalExplnanier = shap.TreeExplainer(classifer)
svm_shap_values = KErnalExplnanier.shap_values(X_test)

In [ ]:
from skmultilearn.problem_transform import ClassifierChain

In [ ]:
#!pip install shap